# Polish MiniLotto Extraction Analysis
This notebook contains code to fetch lotto extraction data, perform frequency analysis, and visualize the results with a color gradient.

## Importing Required Libraries

In [10]:
import requests
import re
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from itertools import combinations
from collections import Counter
import requests
import json
from datetime import datetime

## Function to Fetch Lotto Data

In [11]:
import requests
import json
from datetime import datetime

def read_lotto_data_from_api(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
            'Accept': 'application/json',
            'Referer': 'https://www.lotto.pl/'  # Example, adjust as needed
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()

        lotto_data = {}
        for item in data['items']:
            play_number = item['drawSystemId']
            date_iso = item['drawDate']
            date_object = datetime.fromisoformat(date_iso)
            formatted_date = date_object.strftime('%d.%m.%Y')
            numbers = item['results'][0]['resultsJson']
            game_type = item['gameType']

            lotto_data[play_number] = {
                'date': formatted_date,
                'gametype': game_type,
                'numbers': numbers
            }

        return lotto_data

    except requests.RequestException as e:
        print(f'Error fetching data: {e}')

# URL for the API
api_url = 'https://www.lotto.pl/api/lotteries/draw-results/by-gametype?game=MiniLotto&index=1&size=10000000&sort=drawSystemId&order=DESC'

# Fetching lotto results from the API
lotto_results = read_lotto_data_from_api(api_url)

Error fetching data: 403 Client Error: Forbidden for url: https://www.lotto.pl/api/lotteries/draw-results/by-gametype?game=MiniLotto&index=1&size=10000000&sort=drawSystemId&order=DESC


## Select plays time window

In [12]:
def filter_lotto_results_by_date(lotto_results, start_date, end_date):
    """
    Filters the lotto results to include only those within the specified date range.

    :param lotto_results: Dictionary of lotto results.
    :param start_date: String of start date in 'dd.mm.yyyy' format.
    :param end_date: String of end date in 'dd.mm.yyyy' format.
    :return: Filtered dictionary of lotto results.
    """
    # Convert start_date and end_date to datetime objects
    start_date = datetime.strptime(start_date, "%d.%m.%Y")
    end_date = datetime.strptime(end_date, "%d.%m.%Y")

    # Filter the results
    filtered_results = {}
    for play_number, play_data in lotto_results.items():
        play_date = datetime.strptime(play_data['date'], "%d.%m.%Y")
        if start_date <= play_date <= end_date:
            filtered_results[play_number] = play_data

    return filtered_results

# Example usage
start_date = "01.10.2023"
end_date = "30.12.2023"
lotto_results = filter_lotto_results_by_date(lotto_results, start_date, end_date)

# Now, filtered_lotto_results contains only the plays between the start_date and end_date

AttributeError: 'NoneType' object has no attribute 'items'

## Last 10 draws

In [ ]:
# Displaying the last 10 (newest) results from lotto_results dictionary
last_10_plays = sorted(lotto_results.keys())[-10:]
for play_number in last_10_plays:
    result = lotto_results[play_number]
    print(f"Play Number: {play_number}, Date: {result['date']}, Numbers: {result['numbers']}")


## Frequency Analysis

In [ ]:
def frequency_analysis(lotto_data):
    numbers = []
    for play in lotto_data.values():
        numbers.extend(play['numbers'])
    return Counter(numbers)

number_frequency = frequency_analysis(lotto_results)
sorted_numbers = sorted(number_frequency.items(), key=lambda x: x[1])
numbers, frequencies = zip(*sorted_numbers)

# Define nine thresholds for the deciles
thresholds = np.quantile(frequencies, np.linspace(1/8, 7/8, 7))

# Define 10 colors for each decile
octile_colors = [
    'darkblue',    # Least frequent - coldest
    'blue',
    'lightblue',
    'green',
    'lightgreen',
    'yellow',
    'orange',
    'red'        # Most frequent - hottest
]

# Assign a color based on which decile each frequency falls into
colors = []
for freq in frequencies:
    for i, threshold in enumerate(thresholds):
        if freq <= threshold:
            colors.append(octile_colors[i])
            break
    else:
        colors.append(octile_colors[-1])  # Assign the last color to the top decile

# Plotting the results with the assigned colors
plt.figure(figsize=(15, 5))
plt.bar(numbers, frequencies, color=colors)
plt.xlabel('Number')
plt.ylabel('Frequency')
plt.title('Lotto Number Frequency')
plt.xticks(numbers, rotation=90)
plt.show()

## Print the five more and less probable numbers

In [ ]:
# Sorting the numbers by frequency in descending order
most_frequent_numbers = sorted(number_frequency, key=number_frequency.get, reverse=True)[:5]

# Sorting the numbers by frequency in ascending order
least_frequent_numbers = sorted(number_frequency, key=number_frequency.get)[:5]

print("5 Most Probable Numbers:", most_frequent_numbers)
print("5 Least Probable Numbers:", least_frequent_numbers)

## Verifying if the Most or Least Probable Sets of Five Numbers Have Been Previously Drawn

In [ ]:
def check_occurrence(lotto_data, numbers_set):
    occurrences = []
    for play_number, play_data in lotto_data.items():
        if set(play_data['numbers']) == numbers_set:
            occurrences.append((play_number, play_data['date']))
    return occurrences

# Most and least probable numbers
most_probable_numbers = set(most_frequent_numbers)
least_probable_numbers = set(least_frequent_numbers)

# Check if these sets ever occurred in the history
most_probable_occurrences = check_occurrence(lotto_results, most_probable_numbers)
least_probable_occurrences = check_occurrence(lotto_results, least_probable_numbers)

print("Occurrences of Most Probable Numbers:")
if most_probable_occurrences:
    for play_number, date in most_probable_occurrences:
        print(f"Play Number: {play_number}, Date: {date}")
else:
    print("None")

print("\nOccurrences of Least Probable Numbers:")
if least_probable_occurrences:
    for play_number, date in least_probable_occurrences:
        print(f"Play Number: {play_number}, Date: {date}")
else:
    print("None")


## Comprehensive Analysis of Lotto Number Combinations: Pairs, Triplets, and Quadruplets

In [ ]:
def analyze_combinations(lotto_data, combination_size=2):
    """
    Analyzes the frequency of combinations (like pairs, triplets, quadruplets) in the lotto data.

    :param lotto_data: Dictionary of lotto results.
    :param combination_size: Size of the combinations to analyze (2 for pairs, 3 for triplets, 4 for quadruplets).
    :return: Counter object with the frequency of each combination.
    """
    all_combinations = []

    # Iterate through each draw and find combinations
    for play_data in lotto_data.values():
        numbers = play_data['numbers']
        for combo in combinations(numbers, combination_size):
            all_combinations.append(tuple(sorted(combo)))

    return Counter(all_combinations)

# Example usage
for combination_size in [2, 3, 4]:  # For pairs, triplets, and quadruplets
    print(f"\nAnalyzing {combination_size}-number combinations:")
    combinations_frequency = analyze_combinations(lotto_results, combination_size)

    # Display the most common combinations
    for combo, frequency in combinations_frequency.most_common(10):
        print(f"Combination: {combo}, Frequency: {frequency}")


## Detailed Analysis of Lotto Number Distributions and Trends

Change bin size for broader view or more granularity

In [ ]:
def plot_number_distribution(lotto_data, bin_size=3):
    """
    Plots the distribution of lotto numbers with more detailed x-axis.

    :param lotto_data: Dictionary of lotto results.
    :param bin_size: Size of each bin in the histogram, default is 1 for finer detail.
    """
    all_numbers = []
    for play_data in lotto_data.values():
        all_numbers.extend(play_data['numbers'])

    # Determine the range of numbers
    min_number = min(all_numbers)
    max_number = max(all_numbers)

    # Create a histogram to visualize the distribution
    plt.figure(figsize=(12, 6))
    plt.hist(all_numbers, bins=range(min_number, max_number + bin_size, bin_size), edgecolor='black', alpha=0.7)

    # Setting more x-axis labels
    plt.xticks(range(min_number, max_number + 1, bin_size))

    plt.xlabel('Number')
    plt.ylabel('Frequency')
    plt.title('Detailed Distribution of Lotto Numbers')
    plt.grid(axis='y', alpha=0.75)
    plt.show()

# Example usage
plot_number_distribution(lotto_results)